# exp049
co_visit rank追加

In [1]:
import os
import sys
import traceback
import gc
import random
import pickle
import pathlib
import subprocess
from dataclasses import dataclass
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import inspect

import pandas as pd
import numpy as np
import itertools
import cudf
import lightgbm as lgb
import optuna.integration.lightgbm as optuna_lgb
import matplotlib.pyplot as plt
import seaborn as sns
import line_notify
import my_logger
from noglobal import noglobal

# 設定

In [2]:
@dataclass
class Cfg:
    loglevel = "INFO"
    exp_name = "exp049_hypopt"
    seed = 42
    k = 20
    cand_n = 15
    negative_sample = 1
    train_chunk_n = 1
    type2id = {"clicks":0, "carts":1, "orders":2}
    id2type = {0:"clicks", 1:"carts", 2:"orders"}
    train_weeks = ["week3"]
    valid_week = "week4"
    valid_session_n = 100_000
    input_dir = os.getenv('INPUT_DIR')
    output_dir = os.getenv('OUTPUT_DIR')
    prep_dir = os.getenv("PREP_DIR")

cfg = Cfg()
os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name), exist_ok=True)
os.makedirs(os.path.join(cfg.output_dir, cfg.exp_name, "cache"), exist_ok=True)
random.seed(cfg.seed)

logger = my_logger.init_logger(cfg.exp_name)

In [3]:
params = {'objective': 'binary', 
        'boosting': 'gbdt', 
        'learning_rate': 0.1, 
        'metric': 'binary_logloss', 
        'seed': cfg.seed
        }

In [4]:
for type_ in ["clicks", "carts", "orders"]:
    train = pd.read_pickle(cfg.output_dir + f"exp049/cache/{type_}_train.pkl")
    valid = pd.read_pickle(cfg.output_dir + f"exp049/cache/{type_}_valid.pkl")
    target = "labels"
    not_use_cols = ["session", "aid", target]
    features = [c for c in train.columns if c not in not_use_cols]
    
    tr_x, tr_y = train[features], train[target]
    vl_x, vl_y = valid[features], valid[target]
    tr_data = lgb.Dataset(tr_x, label=tr_y)
    vl_data = lgb.Dataset(vl_x, label=vl_y)

    model = optuna_lgb.train(params, tr_data, valid_sets=[tr_data, vl_data],
                             num_boost_round=2000000, early_stopping_rounds=100, verbose_eval=100)
    logger.info(f"{type_} best_params")
    logger.info(model.params)

[I 2022-12-19 02:38:27,981] A new study created in memory with name: no-name-52834cfc-cfd5-495a-99ac-8b4f02738a55
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/kaggler/.local/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.490492 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.3506	valid_1's binary_logloss: 0.345325
[200]	valid_0's binary_logloss: 0.348191	valid_1's binary_logloss: 0.343654
[300]	valid_0's binary_logloss: 0.346838	valid_1's binary_logloss: 0.342893
[400]	valid_0's binary_logloss: 0.345765	valid_1's binary_logloss: 0.342279
[500]	valid_0's binary_logloss: 0.344773	valid_1's binary_logloss: 0.341766
[600]	valid_0's binary_logloss: 0.343927	valid_

feature_fraction, val_score: 0.339872:  14%|#4        | 1/7 [04:47<28:47, 288.00s/it]

Early stopping, best iteration is:
[1956]	valid_0's binary_logloss: 0.335521	valid_1's binary_logloss: 0.339872
[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.368909 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.351532	valid_1's binary_logloss: 0.345934
[200]	valid_0's binary_logloss: 0.348549	valid_1's binary_logloss: 0.343735
[300]	valid_0's binary_logloss: 0.34729	valid_1's binary_logloss: 0.342962
[400]	valid_0's binary_logloss: 0.346265	valid_1's binary_logloss: 0.342455
[500]	vali

feature_fraction, val_score: 0.339731:  29%|##8       | 2/7 [11:46<30:24, 364.90s/it]

...

Early stopping, best iteration is:
[1392]	valid_0's binary_logloss: 0.331388	valid_1's binary_logloss: 0.339632
[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.421515 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.348169	valid_1's binary_logloss: 0.343572
[200]	valid_0's binary_logloss: 0.345505	valid_1's binary_logloss: 0.341999
[300]	valid_0's binary_logloss: 0.343775	valid_1's binary_logloss: 0.34146
[400]	valid_0's binary_logloss: 0.342347	valid_1's binary_logloss: 0.341097
[500]	vali

regularization_factors, val_score: 0.339283:  35%|###5      | 7/20 [35:45<1:04:21, 297.01s/it]

Early stopping, best iteration is:
[1312]	valid_0's binary_logloss: 0.33215	valid_1's binary_logloss: 0.339754
[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.359079 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.34826	valid_1's binary_logloss: 0.343534
[200]	valid_0's binary_logloss: 0.345563	valid_1's binary_logloss: 0.342001
[300]	valid_0's binary_logloss: 0.343801	valid_1's binary_logloss: 0.341306
[400]	valid_0's binary_logloss: 0.342412	valid_1's binary_logloss: 0.340958
[500]	valid

regularization_factors, val_score: 0.339283:  40%|####      | 8/20 [40:54<1:00:09, 300.79s/it]

Early stopping, best iteration is:
[1548]	valid_0's binary_logloss: 0.330039	valid_1's binary_logloss: 0.339392
[LightGBM] [Info] Number of positive: 2634664, number of negative: 2634664
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.373407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11905
[LightGBM] [Info] Number of data points in the train set: 5269328, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.348169	valid_1's binary_logloss: 0.343572
[200]	valid_0's binary_logloss: 0.345531	valid_1's binary_logloss: 0.342017
[300]	valid_0's binary_logloss: 0.343712	valid_1's binary_logloss: 0.341225
[400]	valid_0's binary_logloss: 0.342193	valid_1's binary_logloss: 0.340757
[500]	val